In [ ]:
import pandas as pd
import os, chardet
import datetime
%load_ext autoreload

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
    return chardet.detect(raw_data)['encoding']

In [ ]:
df=pd.DataFrame()
for f in os.listdir("timesheets"):
    fp=os.path.join("timesheets", f)
    _df=pd.read_csv(fp, encoding=detect_encoding(fp))
    df=pd.concat([df, _df])

# Columns in question
colnames=["Column1", "leave_code", "normal_hrs", "ot_hrs", "ot_type", "leave_hrs"]
fdf=pd.DataFrame()
for i in range(13):
    ncolnames=colnames+[f"tt_label_{i+1}", f"tt_qty_{i+1}"]
    ndf=df[ncolnames]
    ndf.columns=colnames+["label", "qty"]
    fdf=pd.concat([fdf, ndf.reset_index(drop=True)])

# Change data type
fdf.loc[:,"Column1"]=fdf.Column1.apply(lambda dt: datetime.datetime.strptime(dt, "%d/%m/%Y %I:%M:%S %p").date())
fdf.loc[:, "is_weekend"]=fdf.Column1.apply(lambda d: d.strftime("%A") in ["Saturday", "Sunday"])

# Corrected Rates and on calls code
on_calls_code=["On Call MF", "On Call WE", "OnCall PH"]
rates=[
    {
        "start_date": datetime.datetime(year=2024, month=5, day=17).date(),
        "end_date": datetime.datetime(year=2024, month=6, day=14).date(),
        "weekday": 77.2,
        "saturday": 115.8,
        "sunday": 135.1,
        "weekend_pubhol": 202.7,
    },
    {
        "start_date": datetime.datetime(year=2024, month=6, day=14).date(),
        "end_date": datetime.datetime(year=2024, month=6, day=30).date(),
        "weekday": 79.5,
        "saturday": 119.3,
        "sunday": 139.2,
        "weekend_pubhol": 208.8,
    },
    {
        "start_date": datetime.datetime(year=2024, month=7, day=1).date(),
        "end_date": datetime.datetime(year=2025, month=5, day=11).date(),
        "weekday": 81.0,
        "saturday": 121.6,
        "sunday": 141.9,
        "weekend_pubhol": 212.9,
    },
    {
        "start_date": datetime.datetime(year=2025, month=5, day=12).date(),
        "end_date": None,
        "weekday": 83.4,
        "saturday": 125.2,
        "sunday": 146.2,
        "weekend_pubhol": 219.3,
    }
]

fdf=fdf.query("label in @on_calls_code")

In [ ]:
def apply_rates(df: pd.DataFrame, rates: dict):
    ndf=df.copy()
    def apply_rate(rowdat, rate_info):
        # Check if date is inside the rate
        end_date=rate_info["end_date"] if rate_info["end_date"] is not None else datetime.date.today()
        if rowdat.Column1>=rate_info["start_date"] and rowdat.Column1<=end_date:
            if rowdat.label=="On Call MF":
                return rate_info["weekday"]
            elif rowdat.label=="On Call WE":
                return rate_info["saturday"] if rowdat.Column1.weekday()==5 else rate_info["sunday"]
            elif rowdat.label=="OnCall PH":
                return rate_info["sunday"] if rowdat.Column1.weekday()<5 else rate_info["weekend_pubhol"]
        else:
            return False

    all_rates=[]
    for rowdat in df.itertuples():
        rt=False
        for rate in rates:
            if not rt:
                rt=apply_rate(rowdat, rate)
            else: continue
        all_rates.append(rt)

    ndf["rates"]=all_rates
    return ndf

In [ ]:
fdf=apply_rates(fdf, rates)
fdf["total_allowances"]=fdf.qty*fdf.rates
fdf["total_backpay"]=fdf.qty*(fdf.rates-72.2)

# Calculate total backpay outstanding
print(sum(fdf.query("qty>0").total_backpay))